In [91]:
from pycoingecko import CoinGeckoAPI
import datetime as dt
import pandas as pd
import pyodbc

cg = CoinGeckoAPI()

In [92]:
#### TEST
data = cg.get_price(ids='solana', vs_currencies='usd', include_market_cap='true', include_24hr_vol='true', include_24hr_change='true', include_last_updated_at='true')
#dataHist = cg.get_coin_market_chart_range_by_id(id='bitcoin',vs_currency='usd',from_timestamp='1641020400',to_timestamp='1648792799')
#dataHist3 = cg.get_coin_market_chart_by_id(id='bitcoin',vs_currency='usd',days='180')
print(data)

{'solana': {'usd': 34.53, 'usd_market_cap': 11888845656.341112, 'usd_24h_vol': 1405853078.2650146, 'usd_24h_change': 5.081719402689311, 'last_updated_at': 1656645778}}


In [93]:
# Apply a loop to extract the 3 differents crypto 
crypto3 = ['bitcoin','ethereum','maker','solana','aave']
dataHist3=pd.DataFrame()
for i in crypto3:
    dataHist= cg.get_coin_market_chart_range_by_id(id=i,vs_currency='usd',from_timestamp='1641020400',to_timestamp='1648792799')
    dataHist = dataHist ["prices"]
    dataHist = pd.DataFrame(dataHist, columns =['timestampp', 'priceUSD']).sort_values(by='timestampp') 
    dataHist.sort_values(by='timestampp') 
    dataHist.insert(2, "idcoin", i, allow_duplicates=False)
    dataHist3=dataHist3.append(dataHist,ignore_index=True)
    print(dataHist3)

C:\Users\katze\AppData\Local\Temp\ipykernel_74528\1229848024.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataHist3=dataHist3.append(dataHist,ignore_index=True)
C:\Users\katze\AppData\Local\Temp\ipykernel_74528\1229848024.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataHist3=dataHist3.append(dataHist,ignore_index=True)


         timestampp      priceUSD   idcoin
0     1641020554453  47128.848463  bitcoin
1     1641024444418  47292.778947  bitcoin
2     1641028659948  47251.883398  bitcoin
3     1641031594501  47223.103990  bitcoin
4     1641035011717  46983.080704  bitcoin
...             ...           ...      ...
2155  1648774924255  45557.572208  bitcoin
2156  1648778560739  45086.747368  bitcoin
2157  1648782128779  44347.204444  bitcoin
2158  1648785652514  44543.548925  bitcoin
2159  1648789316372  44719.711080  bitcoin

[2160 rows x 3 columns]
         timestampp      priceUSD    idcoin
0     1641020554453  47128.848463   bitcoin
1     1641024444418  47292.778947   bitcoin
2     1641028659948  47251.883398   bitcoin
3     1641031594501  47223.103990   bitcoin
4     1641035011717  46983.080704   bitcoin
...             ...           ...       ...
4314  1648774944046   3299.609756  ethereum
4315  1648778506553   3267.096946  ethereum
4316  1648782024309   3220.063718  ethereum
4317  1648785681924

C:\Users\katze\AppData\Local\Temp\ipykernel_74528\1229848024.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataHist3=dataHist3.append(dataHist,ignore_index=True)


         timestampp      priceUSD   idcoin
0     1641020554453  47128.848463  bitcoin
1     1641024444418  47292.778947  bitcoin
2     1641028659948  47251.883398  bitcoin
3     1641031594501  47223.103990  bitcoin
4     1641035011717  46983.080704  bitcoin
...             ...           ...      ...
6474  1648775207890   2111.844830    maker
6475  1648778821472   2071.835995    maker
6476  1648782417402   2033.947412    maker
6477  1648786029311   2062.610767    maker
6478  1648789602261   2059.319285    maker

[6479 rows x 3 columns]
         timestampp      priceUSD   idcoin
0     1641020554453  47128.848463  bitcoin
1     1641024444418  47292.778947  bitcoin
2     1641028659948  47251.883398  bitcoin
3     1641031594501  47223.103990  bitcoin
4     1641035011717  46983.080704  bitcoin
...             ...           ...      ...
8634  1648774919232    124.446054   solana
8635  1648778527504    121.895624   solana
8636  1648782181581    118.668438   solana
8637  1648785697554    120.44

C:\Users\katze\AppData\Local\Temp\ipykernel_74528\1229848024.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataHist3=dataHist3.append(dataHist,ignore_index=True)
C:\Users\katze\AppData\Local\Temp\ipykernel_74528\1229848024.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataHist3=dataHist3.append(dataHist,ignore_index=True)


In [94]:
print(dataHist3)

          timestampp      priceUSD   idcoin
0      1641020554453  47128.848463  bitcoin
1      1641024444418  47292.778947  bitcoin
2      1641028659948  47251.883398  bitcoin
3      1641031594501  47223.103990  bitcoin
4      1641035011717  46983.080704  bitcoin
...              ...           ...      ...
10792  1648774916773    216.931351     aave
10793  1648778444415    210.760224     aave
10794  1648782022535    209.194691     aave
10795  1648785642123    216.039156     aave
10796  1648789297353    219.784522     aave

[10797 rows x 3 columns]


In [95]:
# Convert entire DataFrame to string
dataHist3=dataHist3.applymap(str)
print(dataHist3.dtypes)

timestampp    object
priceUSD      object
idcoin        object
dtype: object


In [96]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=FREYJA\SQLEXPRESS;'
                      'Database=tradingCrypto;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM [dbo].[cryptoPrices_raw]')

for i in cursor:
    print(i)

In [97]:
# Insert Dataframe into SQL Server:
for index, row in dataHist3.iterrows():
     cursor.execute("INSERT INTO dbo.cryptoPrices_raw (timestampp,id_coin,priceUSD) values(?,?,?)", row.timestampp, row.idcoin, row.priceUSD)
conn.commit()
cursor.close()